# PySentimiento: Análisis de Opinión y NLP Social Multilingüe


`pysentimiento` es una biblioteca de Python que ofrece modelos preentrenados para tareas de procesamiento de lenguaje natural centradas en redes sociales. Es compatible con varios idiomas (español, inglés, portugués, italiano) y permite realizar análisis de sentimiento, detección de emociones, discurso de odio, ironía, análisis de entidades nombradas (NER), y etiquetas gramaticales (POS).

A continuación, se muestran ejemplos de uso con texto en español.


In [ ]:

# Instalar pysentimiento (si no está instalado)
!pip install pysentimiento


## Flujo: del texto a la predicción

Para entender cómo funciona internamente la herramienta, aquí se describe el pipeline típico que ocurre al llamar `analyzer.predict(text)`:

### 1. Preprocesamiento
El texto crudo se pasa primero por el preprocesador de tweets (a menos que esté deshabilitado). Este paso limpia el texto aplicando reglas como la normalización de menciones y URLs, la reducción de repeticiones de caracteres, la conversión de emojis a texto, etc. El resultado es una versión normalizada del texto de entrada.

### 2. Tokenización
El tokenizador del analizador (una subclase de `AutoTokenizer` de Hugging Face) convierte el texto preprocesado en una secuencia de tokens/IDs que el modelo Transformer puede interpretar. Esto incluye dividir el texto en subpalabras y agregar tokens especiales (como `[CLS]` y `[SEP]` para BERT, o solo tokens de inicio/fin en el caso de RoBERTa). También se realiza padding o truncamiento según la longitud máxima permitida.

### 3. Inferencia del Modelo
Los IDs de tokens se envían al modelo Transformer (que internamente es un modelo de PyTorch). Dependiendo de la tarea:

- **Tareas de clasificación:** El modelo produce un conjunto de logits (puntuaciones no normalizadas) para cada clase. Por ejemplo, el modelo de sentimiento devuelve tres logits (uno por cada clase: POS, NEG, NEU).
- **Tareas multi-etiqueta:** El modelo devuelve un logit por cada etiqueta posible (por ejemplo, tres logits para las etiquetas de discurso de odio: `hateful`, `targeted`, `aggressive`).
- **Tareas de etiquetado secuencial (NER/POS):** Se devuelve un logit por token por cada posible etiqueta.

### 4. Post-procesamiento de la Predicción
Los logits producidos por el modelo se convierten en probabilidades:

- Para clasificación de una sola etiqueta, se aplica **softmax**, y se selecciona la etiqueta con mayor probabilidad como `output`.
- Para clasificación multi-etiqueta (como discurso de odio), se aplica **sigmoid** a cada logit, y se seleccionan las etiquetas cuya probabilidad supera un umbral (por defecto, 0.5).


Luego, las etiquetas seleccionadas se ensamblan en un objeto 'resultado'.

### 5. Formato de Salida
El resultado se devuelve como un objeto `AnalyzerOutput` (o una lista si se procesan múltiples textos). Este objeto contiene:

- `output`: la etiqueta predicha (para tareas de una sola etiqueta) o una lista de etiquetas (para tareas multi-etiqueta). En tareas secuenciales como NER o POS, puede ser una lista de anotaciones por token.
- `probas`: un diccionario que asigna a cada etiqueta su probabilidad estimada.  
  Por ejemplo:
  ```python
  {'POS': 0.998, 'NEG': 0.002, 'NEU': 0.000}
  ```
  para una oración claramente positiva, o:
  ```python
  {'hateful': 0.987, 'targeted': 0.978, 'aggressive': 0.969}
  ```
  ppara un texto con discurso de odio.

## Veamos algunos Ejemplos de funcionalidades

Tras la instalación, el uso de la biblioteca es muy sencillo. 

Se crea un analizador para la tarea y el idioma de interés y, a continuación, se llama a la funcion 'predict' con nuevos textos. 

En este caso definimos como tarea el análisis de sentimiento y el idioma que queremos usar como español


In [ ]:

from pysentimiento import create_analyzer

# Analizador de sentimiento en español
sentiment_analyzer = create_analyzer(task="sentiment", lang="es")


In [ ]:

texto_positivo = "Qué gran jugador es Messi"
texto_negativo = "Esto es pésimo"
texto_neutro = "El evento será a las 5 PM"

print("Positivo:", sentiment_analyzer.predict(texto_positivo))
print("Negativo:", sentiment_analyzer.predict(texto_negativo))
print("Neutro:", sentiment_analyzer.predict(texto_neutro))


## Emotion Analysis

`pysentimiento` provee análisis de emociones a través de modelos pre-entrenados con los datasets de [EmoEvent](https://github.com/fmplaza/EmoEvent-multilingual-corpus/)

In [ ]:

emotion_analyzer = create_analyzer(task="emotion", lang="es")

print("Emoción:", emotion_analyzer.predict("Estoy muy feliz con el resultado"))


## Hate Speech
Replicamos la operación pero esta ver para análissi de odio

In [ ]:

hate_analyzer = create_analyzer(task="hate_speech", lang="es")

texto_odio = "Vaya guarra barata y de poca monta es XXXX!"
print("Discurso de odio:", hate_analyzer.predict(texto_odio))


In [ ]:

irony_analyzer = create_analyzer(task="irony", lang="es")

texto_ironia = "Sí claro, como si el gobierno alguna vez resolviera algo..."
print("Ironía:", irony_analyzer.predict(texto_ironia))


## Token Labeling tasks


`pysentimiento` cuenta con analizadores para POS tagging & NER gracias al dataset multilingual [LinCE](https://ritual.uh.edu/lince/)

In [ ]:

from pysentimiento.preprocessing import preprocess_tweet

texto_tweet = "@usuario deberías ver esto http://bit.ly/ejemplo jajajajajaja"
print("Texto original:", texto_tweet)
print("Preprocesado:", preprocess_tweet(texto_tweet))


In [ ]:

ner_analyzer = create_analyzer(task="ner", lang="es")

print("Entidades nombradas:", ner_analyzer.predict("Lionel Messi juega en el Inter de Miami"))


In [ ]:

pos_analyzer = create_analyzer(task="pos", lang="es")

print("POS tagging:", pos_analyzer.predict("Messi corre rápidamente con el balón"))



## 🧠 Ejercicio: Análisis de Sentimientos y Emociones en Opiniones de Usuarios

### 📝 Objetivo
Usar `pysentimiento` para analizar una serie de opiniones extraídas de redes sociales o reseñas, y comparar los resultados obtenidos con la intuición humana. Reflexionar sobre aciertos, errores y posibles fuentes de ambigüedad.

### 📦 Materiales
- Lista de 10 a 15 frases que simulen opiniones de usuarios (pueden ser reales o inventadas).
- Un cuaderno (notebook) en Colab o Jupyter con `pysentimiento` instalado.

### 🔧 Instrucciones

1. **Preparación del Dataset**




In [ ]:
opiniones = [
       "Me encantó la atención al cliente, volvería sin dudarlo.",
       "Una pérdida de tiempo, el producto no sirve.",
       "El hotel estaba bien, aunque el desayuno era mejorable.",
       "No sé qué pensar, fue una experiencia rara.",
       "¡Excelente servicio y muy rápido!",
       "Demasiado caro para lo que ofrece.",
       "No me gustó, pero entiendo que a otros sí.",
       "La película fue una joya, me hizo llorar de emoción.",
       "Malisimo, no lo recomiendo a nadie.",
       "La app está bien diseñada, pero falla mucho."
   ]


## 2. Predicción automática con PySentimiento


In [ ]:
from pysentimiento import create_analyzer

sentiment_analyzer = create_analyzer(task="sentiment", lang="es")
emotion_analyzer = create_analyzer(task="emotion", lang="es")

for frase in opiniones:
    sentimiento = sentiment_analyzer.predict(frase)
    emocion = emotion_analyzer.predict(frase)
    print(f"Frase: {frase}")
    print(f"  ➤ Sentimiento: {sentimiento.output}, probabilidades: {sentimiento.probas}")
    print(f"  ➤ Emoción: {emocion.output}, probabilidades: {emocion.probas}")
    print("-" * 80)


### Tareas a realizar

🧐 Clasificar cada frase como positiva, negativa o neutra antes de ejecutar el modelo. Luego comparalo con la predicción automática.

❓ ¿Coincide tu intuición con la del modelo? ¿En qué casos no? ¿Por qué podría fallar el modelo?

💡 Observar los casos donde el modelo predice neutral. ¿Te parece apropiado? ¿Qué tipo de lenguaje aparece allí?

🔍 Revisar también las emociones detectadas. ¿Hay ambigüedad? ¿Captura bien la emoción dominante?

🧪 Modificar algunas frases para ver cómo cambian las predicciones. Por ejemplo: "El servicio fue increíblemente lento" vs. "El servicio fue lento, pero aceptable".

## Reflexión Final

¿Qué fortalezas y limitaciones encontraste en el modelo?

¿Qué tipos de sesgos podrían surgir si solo usamos este tipo de modelos para tomar decisiones?

¿Cómo mejorarías este pipeline para un caso de uso real, como popr ej. para  monitorear opiniones sobre una marca?

   
Para más información: [https://github.com/pysentimiento/pysentimiento](https://github.com/pysentimiento/pysentimiento)
